# Eddy Viscosity Models and Solver Review

**Here's a brief review of the solver and the eddy viscosity models:**

-  This notebook includes an eddy viscosity model where the *eddy viscosity coefficient* can be dynamically adjusted.


## 2D Turbulence

We solve Eqs. in the vorticity-streamfunction, $\omega-\psi$, formulation, where

With this formulation, the governing equations are

$$
\begin{aligned}
\nabla^2\psi &= - \omega, \tag{1} \\
\frac{\partial \omega}{\partial t} + \mathcal{N}\left(\omega,\psi\right) &= \frac{1}{Re}\nabla^2\omega -\chi \omega - f + \beta v,
\end{aligned}
$$

where $\mathcal{N}(\omega,\psi)$ is 

$$
\mathcal{N}(\omega,\psi) = \frac{\partial \omega}{\partial x}\frac{\partial \psi}{\partial y} - \frac{\partial \omega}{\partial y}\frac{\partial \psi}{\partial x}. 
$$

$f$ is a deterministic forcing \cite{chandler2013invariant, kochkov2021machine}:

$$
f = f_{k_x}\cos(f_{k_x}x) + f_{k_y}\cos(f_{k_y}y), \tag{4}
$$

where $f_{k_{x}}$ and $f_{k_{y}}$ are the forcing wavenumbers, $\chi$ represents the Rayleigh drag coefficient, $\beta$ represents the effect of beta-plane turbulence, and $\bm{u}=(u,v)$ is the velocity. In 2D, the velocity can be represented by stream function

$$
u = \frac{\partial \psi}{\partial y}, v = -\frac{\partial \psi}{\partial x},
$$

In DNS, governing equations are solved in a doubly periodic domain using a Fourier-Fourier pseudo-spectral solver with second-order Adams-Bashforth and Crank Nicholson time-integration schemes for the advection and viscous terms, respectively (time step $\Delta t_{\text{DNS}}$).

For LES, we use the same solver with lower spatio-temporal resolution: We use $N_{\rm{LES}}$ that is 8 to 64 times smaller than $N_{\text{DNS}}$, and $\Delta t_{\text{LES}}=10\Delta t_{\text{DNS}}$.

For LES, equations are derived by applying a homogenous filter to leading to,

$$
\begin{aligned}
    \nabla^2\overline{\psi} &= -\overline{\omega}, \\
    \frac{\partial \overline{\omega}}{\partial t} + \mathcal{N}(\overline{\omega},\overline{\psi}) &= \frac{1}{Re} \nabla^2\overline{\omega} -\chi\overline{\omega} - \overline{f} -\left(\overline{\mathcal{N}(\omega,\psi)}-\mathcal{N}(\overline{\omega},\overline{\psi}) \right), 
\end{aligned}
$$

$$
\begin{aligned}
    \Pi_{\omega} &= \nabla \times \nabla \cdot \bm{\tau} \\
    &= \overline{\mathcal{N}(\omega,\psi)}-\mathcal{N}(\overline{\omega},\overline{\psi})
\end{aligned}
$$


## Eddy Viscosity Models
$$
\Pi_{\omega} = - \nu_e \nabla^2 \overline{\omega},
$$
where $\nu$ is the eddy viscosity needs to be modeled

### Smagorinsky (SMAG)
$$
\nu_e = (C_s\Delta)^2 |\overline{\mathcal{S}}| \\
\overline{\mathcal{S}} = \sqrt{2 \overline{S}_{ij} \overline{S}_{ij}} \\
\overline{S}_{ij} = \frac{1}{2} \left(\frac{\partial \overline{u}_i}{\partial x_j} + \frac{\partial \overline{u}_j}{\partial x_i} \right)
$$
Here $C_s$ is the SMAG coefficient or eddy viscosity coefficient and $\Delta$ is the filter width.

### LEITH
$$
\nu_e = (C_L\Delta)^3 |\nabla \overline{\omega}| \\
$$
Here $C_L$ is the LEITH coefficient or eddy viscosity coefficient. 

In [39]:
# Import the Py2D_solver function from the py2d.Py2D_solver module
from py2d.Py2D_solver import Py2D_solver
from py2d.datamanager import gen_path, get_last_file
from py2d.filter import filter2D_2DFHIT

# Import the numpy library for numerical operations
import numpy as np
import os

from scipy.io import loadmat, savemat

# system parameters
Re = 20e3
fkx = 4
fky = 4
alpha = 0.1
beta = 0

# set parameters for high_res simulation
dt_high_res = 5e-5
NX_high_res = 1024

# set parameters for low_res simulation
dt_low_res = 10*dt_high_res
NX_low_res = 64

# Set an initial value for the eddy viscosity coefficient
eddyViscosityCoeff_temp = 0.17

# Set the time interval to update the eddy viscosity coefficient
t_update_eddy_viscosity = 10*dt_low_res

# The high resolution simulation and the low resolution simulation are runnning simultaneously

for count_update_eddy_viscosity_parameter in range(5):

    for count_low_res in range(int(t_update_eddy_viscosity//dt_low_res)):

    ############################# High Res Simulation #############################

        # Running high resolution simulation for length of time = dt_low_res

        if count_low_res == 0 and count_update_eddy_viscosity_parameter == 0:
            # Initialize a flag to control whether to resume a simulation
            resumeSimulation = False
        else:
            resumeSimulation = True

        Omega_high_res = Py2D_solver(
                Re=Re,  # Reynolds number
                fkx=fkx,  # Forcing wavenumber in x
                fky=fky,  # Forcing wavenumber in y
                alpha=alpha,  # Rayleigh drag coefficient
                beta=beta,  # Coriolis parameter
                NX=NX_high_res,  # Number of grid points in x and y (options: 32, 64, 128, 256, 512)
                SGSModel_string='NoSGS',  # SGS model to use (options: 'NoSGS', 'SMAG', 'DSMAG', 'LEITH', etc.)
                eddyViscosityCoeff=0,  # Coefficient for eddy viscosity models
                dt=dt_high_res,  # Time step
                saveData=True,  # Flag to save data
                tSAVE=dt_high_res,  # Time interval to save data
                tTotal=dt_low_res,  # Total time of simulation
                readTrue=False, 
                ICnum=1,  # Initial condition number (options: 1 to 20)
                direct_IC=None,
                resumeSim=resumeSimulation,  # Flag to start a new simulation or resume an existing one
            )

        ## Example code to last 10 snapshots of the Vorticity  (Omega) data from the high resolution simulation

        # Get the path to the directory where the data is saved
        _, SAVE_DIR_DATA_high_res, SAVE_DIR_IC_high_res = gen_path(NX=NX_high_res, dt=dt_high_res, ICnum=1, Re=Re, fkx=fkx, fky=fky, alpha=alpha, beta=beta, SGSModel_string='NoSGS')

        # Individual file contains data for single snapshot with files numbered in the asceding order of saving
        # last_file_number_data is the latest saved file number
        last_file_number_data_high_res = get_last_file(SAVE_DIR_DATA_high_res)

        Omega_high_res_snapshots = []
        for count in range(10):
            filenumber = last_file_number_data_high_res - count
            filename = f"{SAVE_DIR_DATA_high_res}{filenumber}.mat"

            data = loadmat(filename)

            # Last 10 snapshots are saved in the following array
            Omega_high_res_snapshots.append(data['Omega'])

    ############################# Low Res Simulation #############################

        # Running low resolution simulation for single time step

        if count_low_res == 0 and count_update_eddy_viscosity_parameter == 0:
            direct_IC_low_res = None
            resumeSimulation_low_res = False
        else:
            # The initial conditions of the low resolution simulation are taken from the high resolution simulation
            # The solver uses Adam-Bashforth scheme to update the solution in time for the non-linear convection term,
            # and the solution at the previous time step (dt_low_res) is required for this scheme.
                
            # For the given example I am using the last snapshot of the high resolution simulation as the initial condition for the low resolution simulation
            # Coarse graining the last snapshot of the high resolution simulation to the low resolution simulation grid
            Omega_low_res_initial = filter2D_2DFHIT(Omega_high_res, filterType='gaussian', coarseGrainType='spectral', Ngrid=[NX_low_res, NX_low_res])

            # Using the last snapshot of the high resolution simulation as the initial condition for the low resolution simulation. 
            # Using Omega for two consecutive timesteps (following are placeholder values)
            direct_IC_low_res = {
            'Omega1_hat': np.fft.fft2(Omega_low_res_initial),
            'Omega0_hat': np.fft.fft2(Omega_low_res_initial),
            }
            resumeSimulation_low_res = True

        Omega_low_res = Py2D_solver(
                Re=Re,  # Reynolds number
                fkx=fkx,  # Forcing wavenumber in x
                fky=fky,  # Forcing wavenumber in y
                alpha=alpha,  # Rayleigh drag coefficient
                beta=beta,  # Coriolis parameter
                NX=NX_low_res,  # Number of grid points in x and y (options: 32, 64, 128, 256, 512)
                SGSModel_string='LEITH',  # SGS model to use (options: 'NoSGS', 'SMAG', 'DSMAG', 'LEITH', etc.)
                eddyViscosityCoeff=eddyViscosityCoeff_temp,  # Coefficient for eddy viscosity models
                dt=dt_low_res,  # Time step
                saveData=True,  # Flag to save data
                tSAVE=dt_low_res,  # Time interval to save data
                tTotal=dt_low_res,  # Total time of simulation
                readTrue=False, 
                ICnum=1,  # Initial condition number (options: 1 to 20)
                direct_IC=direct_IC_low_res,
                resumeSim=resumeSimulation_low_res,  # Flag to start a new simulation or resume an existing one
            )


    ############################## Update Eddy Viscosity ##############################
        
    # Calculate the eddy viscosity coefficient using the last 10 snapshots low resolution simulation

    # Get the path to the directory where the data is saved
    _, SAVE_DIR_DATA_low_res, SAVE_DIR_IC_low_res = gen_path(NX=NX_low_res, dt=dt_low_res, ICnum=1, Re=Re, fkx=fkx, fky=fky, alpha=alpha, beta=beta, SGSModel_string='LEITH')


    # Individual file contains data for single snapshot with files numbered in the asceding order of saving
    # last_file_number_data is the latest saved file number
    last_file_number_data_low_res = get_last_file(SAVE_DIR_DATA_low_res)

    Omega_low_res_snapshots = []
    for count in range(10):
        filenumber = last_file_number_data_low_res - count
        filename = f"{SAVE_DIR_DATA_low_res}{filenumber}.mat"

        data = loadmat(filename)

        # Last 10 snapshots are saved in the following array
        Omega_low_res_snapshots.append(data['Omega'])

    # Omega_low_res_snapshots can be used to update the eddy viscosity coefficient
    eddyViscosityCoeff_temp +=  0.01
    print('*******************************************************************************************')
    print(f"Updated eddy viscosity coefficient = {eddyViscosityCoeff_temp}")
    print('*******************************************************************************************')



#     # Just keep the last 10 snapshots of the high resolution simulation Delete the rest
#     for count in range(10):
#         filenumber = last_file_number_data - count - 1
#         filename = f"{SAVE_DIR_DATA_high_res}{filenumber}.mat"
#         os.remove(filename)


[Errno 17] File exists: 'results/Re20000_fkx4fky4_r0.1_b0/NoSGS/NX1024/dt5e-05_IC1/data/'
+--------------------------------------------------------------------+--------+
|                         Run Configuration                          | Value  |
+--------------------------------------------------------------------+--------+
|                           Time Step (dt)                           | 5e-05  |
|                         Resume Simulation                          | False  |
| Read Initialization (readTrue), If False: Will read IC from a file | False  |
|                      Saving Data  (saveData)                       |  True  |
|               Save data every t th timestep (tSAVE)                | 5e-05  |
|               Save data every Nth iteration (NSAVE)                |   1    |
|                   Length of simulation (tTotal)                    | 0.0005 |
|                Maximum Number of Iterations (maxit)                |   10   |
+-----------------------------

In [43]:
## Deleting files to save space. Keeping the last 10 snapshots of high and low res

import os

def remove_files_range(save_dir, last_file_number):
    # Generate file names using range and list comprehension
    filenames = [f"{save_dir}{i}.mat" for i in range(last_file_number - 10)]

    # Remove each file
    for filename in filenames:
        if os.path.exists(filename):
            os.remove(filename)

# Usage
remove_files_range(SAVE_DIR_IC_high_res, last_file_number_data_high_res)
remove_files_range(SAVE_DIR_DATA_high_res, last_file_number_data_high_res)
remove_files_range(SAVE_DIR_IC_low_res, last_file_number_data_low_res)
remove_files_range(SAVE_DIR_DATA_low_res, last_file_number_data_low_res)

Pseudo code

- run high res simulation. Length of high res simulation = single dt_low_res
- run low res simulation for single timestep = dt_low_res. The initial condition for low res simulation can be obtained from high res data
- modify the eddy viscosity coefficient after running both the high_res and the low res for total time of t_update_eddy_viscosity